In [2]:
# Import PyTorch libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image, ImageOps
import os

# Other libraries we'll use
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

print("Libraries imported - ready to use PyTorch", torch.__version__)

Libraries imported - ready to use PyTorch 2.7.0+cu128


In [3]:
# The images are in the data/shapes folder
data_path = '/mnt/d/RuhunaNew/Academic/Research/Facial_Recog/Group_50/SampleDataset/train'

# Get the class names
classes = os.listdir(data_path)
classes.sort()
print(len(classes), 'classes:')
print(classes)

9 classes:
['angry', 'boring', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'stress', 'suprise']


In [4]:
#print the image count for each class
for c in classes:
    print('{}: {} images'.format(c, len(os.listdir(os.path.join(data_path, c)))))

angry: 1000 images
boring: 1000 images
disgust: 1000 images
fear: 1000 images
happy: 1000 images
neutral: 1000 images
sad: 1000 images
stress: 1000 images
suprise: 1000 images


In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__() # Super is indeed super
        
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=(5,5),bias=True)
        # for single grayscale image use 6 different kernls of size (5,5) to produce 6 diff feature maps 
        
        self.conv_2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=(3,3))
        # from the existing 6 already feat maps, use 12 different kernal filters of size (3,3) to get TOTAL of 12 new feat
        
        self.dense_1 = nn.Linear(in_features=12*4*4,out_features=128) # WHYY (12*4*4) it is explained in the end
        # Flatten the output of conv_2d in 12*4*4
        
        self.fc_2 = nn.Linear(in_features=128,out_features=64)
        # Fully Connected = fc_2 = Dense Layer = dense_2
        
        self.out = nn.Linear(in_features=64,out_features=9)
        # output layer. Output number of neurons = num of classes for classification & 1 for regression
        
    
    def forward(self,t):
        '''
        implement a forward pass on a Tensor 't' of rank 'R'
        '''
        # input  layer 1 though it is never needed
        # t = t
        
        # second layer. Layer is a mix of functions that has weights 
        t = self.conv_1(t) # works by calling __call__() method inside class
        t = F.relu(input=t,) # it is not a layer but Function (layers have weights, activations don't)
        t = F.max_pool2d(t,kernel_size=(3,3),stride=2) # max pooling
        
        # third layer
        t = self.conv_2(t) # works by calling __call__() method inside class
        t = F.relu(input=t,) # it is not a leyer but Function as layers have weights
        t = F.max_pool2d(t,kernel_size=(3,3),stride=2) # max pool
        
        # fourth layer
        t = t.reshape(-1,12*4*4) 
        # due to Conv and pooling operations, our image has been reduced from (1,28,28) to (4,4)
        # use ((input_size - filter_size + 2*padding)/stride )+1  for each  cov and max_pool 
        # it assumes input and kernel size are square
        t = self.dense_1(t)
        t = F.relu(t)
        
        # Fifth layer
        t = self.fc_2(t)
        t = F.relu(t)
        
        # output
        t = self.out(t)
        # t = F.softmax(t,dim=1)
        # commented because loss function used will be cross_entropy which has softmax behind the scenes
        return t